# This app focuses on extracting cryptocurrencies data from coingecko's API

## APIs used:

    - Get/coins/markets to get a list of all supported coins, ath, price, volume and other info
    - Get/coins/{id}/history to get community and developer data


In [1]:
import requests
import pandas as pd

In [2]:
# Find an API that has information on coin
base_url = "https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=1&sparkline=false"
print(base_url)

# Request json file from the url
response_json = requests.get(base_url).json()

# checkou response
response_json

https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=300&page=1&sparkline=false


[{'id': 'bitcoin',
  'symbol': 'btc',
  'name': 'Bitcoin',
  'image': 'https://assets.coingecko.com/coins/images/1/large/bitcoin.png?1547033579',
  'current_price': 49533,
  'market_cap': 926409841338,
  'market_cap_rank': 1,
  'fully_diluted_valuation': 1040954156538,
  'total_volume': 98429439366,
  'high_24h': 55354,
  'low_24h': 47882,
  'price_change_24h': -5508.97346679,
  'price_change_percentage_24h': -10.00864,
  'market_cap_change_24h': -103564242740.03442,
  'market_cap_change_percentage_24h': -10.05503,
  'circulating_supply': 18689206.0,
  'total_supply': 21000000.0,
  'max_supply': 21000000.0,
  'ath': 64805,
  'ath_change_percentage': -23.50982,
  'ath_date': '2021-04-14T11:54:46.763Z',
  'atl': 67.81,
  'atl_change_percentage': 73001.27793,
  'atl_date': '2013-07-06T00:00:00.000Z',
  'roi': None,
  'last_updated': '2021-04-23T13:03:04.065Z'},
 {'id': 'ethereum',
  'symbol': 'eth',
  'name': 'Ethereum',
  'image': 'https://assets.coingecko.com/coins/images/279/large/ethe

In [6]:
# Put the response into a data frame
crypto_df = pd.DataFrame(response_json)
crypto_df.head()

,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,...,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,49533.000000,926409841338,1,1.040954e+12,9.842944e+10,55354.00,...,2.100000e+07,21000000.0,64805.00,-23.50982,2021-04-14T11:54:46.763Z,67.810000,7.300128e+04,2013-07-06T00:00:00.000Z,None,2021-04-23T13:03:04.065Z
1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,2258.880000,261075398474,2,NaN,7.588274e+10,2640.37,...,NaN,NaN,2640.37,-14.45530,2021-04-22T16:34:50.091Z,0.432979,5.215641e+05,2015-10-20T00:00:00.000Z,"{'times': 59.7334658951109, 'currency': 'btc',...",2021-04-23T13:03:01.248Z
2,binancecoin,bnb,Binance Coin,https://assets.coingecko.com/coins/images/825/...,506.430000,78044063834,3,8.612454e+10,1.109119e+10,579.11,...,1.705337e+08,170533651.9,610.06,-17.21578,2021-04-12T09:32:32.635Z,0.039818,1.268255e+06,2017-10-19T00:00:00.000Z,None,2021-04-23T13:02:39.834Z
3,ripple,xrp,XRP,https://assets.coingecko.com/coins/images/44/l...,1.090000,50621300430,4,NaN,2.491884e+10,1.40,...,1.000000e+11,NaN,3.40,-67.80619,2018-01-07T00:00:00.000Z,0.002686,4.062994e+04,2014-05-22T00:00:00.000Z,None,2021-04-23T13:04:11.598Z
4,tether,usdt,Tether,https://assets.coingecko.com/coins/images/325/...,0.998849,49520549694,5,NaN,2.115782e+11,1.01,...,4.942289e+10,NaN,1.32,-24.27030,2018-07-24T00:00:00.000Z,0.572521,7.501123e+01,2015-03-02T00:00:00.000Z,None,2021-04-23T13:03:03.931Z


In [7]:
# Clean data by dropping columns
crypto_df = crypto_df[['id', 'symbol', 'current_price','market_cap', 'market_cap_rank', 'ath', 'ath_change_percentage', 'ath_date','atl', 'atl_change_percentage', 'atl_date']]
crypto_df['market_cap'] = crypto_df['market_cap']/1000000000

# Rename columns to see easier
crypto_df.rename(columns={'market_cap':'market_cap_billion(2021)',
                          'current_price' : 'price_04-22-21'
                          
                         }, inplace=True)
crypto_df.head()

,id,symbol,price_04-22-21,market_cap_billion(2021),market_cap_rank,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date
0,bitcoin,btc,49533.000000,926.409841,1,64805.00,-23.50982,2021-04-14T11:54:46.763Z,67.810000,7.300128e+04,2013-07-06T00:00:00.000Z
1,ethereum,eth,2258.880000,261.075398,2,2640.37,-14.45530,2021-04-22T16:34:50.091Z,0.432979,5.215641e+05,2015-10-20T00:00:00.000Z
2,binancecoin,bnb,506.430000,78.044064,3,610.06,-17.21578,2021-04-12T09:32:32.635Z,0.039818,1.268255e+06,2017-10-19T00:00:00.000Z
3,ripple,xrp,1.090000,50.621300,4,3.40,-67.80619,2018-01-07T00:00:00.000Z,0.002686,4.062994e+04,2014-05-22T00:00:00.000Z
4,tether,usdt,0.998849,49.520550,5,1.32,-24.27030,2018-07-24T00:00:00.000Z,0.572521,7.501123e+01,2015-03-02T00:00:00.000Z
